# Notebook 01: Carregamento e Exploração de Dados Médicos

Este notebook é o primeiro passo do pipeline RAG para dados médicos. Aqui vamos:

1. **Carregar** o dataset `ori_pqal.json` do PubMedQA
2. **Explorar** a estrutura dos dados (QUESTION, CONTEXTS, LONG_ANSWER, MESHES)
3. **Analisar** estatísticas básicas do dataset
4. **Visualizar** exemplos de entradas

## 📋 Pré-requisitos

- Arquivo `ori_pqal.json` disponível no caminho configurado
- Variáveis de ambiente configuradas (veja `.env.example`)
- Dependências instaladas: `pip install -r requirements.txt`

## 🔄 Ordem de Execução

Execute as células **sequencialmente** (de cima para baixo) para garantir que todas as dependências estejam carregadas corretamente.


In [5]:
# ============================================================================
# ETAPA 0: IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÕES
# ============================================================================
# Esta célula importa todas as bibliotecas necessárias e configura o ambiente
# Execute esta célula PRIMEIRO antes de qualquer outra operação

import sys
import importlib
from pathlib import Path

# Adiciona o diretório raiz ao path para importar módulos
# Quando executado do diretório notebooks/, o parent é rag_medical/
root_dir = Path.cwd().parent
if str(root_dir) not in sys.path:
    sys.path.insert(0, str(root_dir))

# Recarrega módulos se já foram importados (útil durante desenvolvimento)
if 'config.settings' in sys.modules:
    importlib.reload(sys.modules['config.settings'])
if 'config' in sys.modules:
    importlib.reload(sys.modules['config'])

# Importa módulos do pipeline RAG
from scripts.data_loader import (
    load_medical_dataset,
    validate_dataset_structure,
    get_dataset_stats
)
from config.settings import Settings, get_settings

# Carrega configurações
settings = get_settings()

# Valida configurações (strict=False permite exploração sem Pinecone/Embeddings)
is_valid, errors = settings.validate(strict=False)
if not is_valid:
    print("❌ ERROS DE CONFIGURAÇÃO:")
    for error in errors:
        print(f"   - {error}")
    print("\n💡 Configure as variáveis de ambiente no arquivo .env")
    raise ValueError("Configurações inválidas")
else:
    print("✅ Configurações validadas com sucesso!")
    print(f"📁 Arquivo de dados: {settings.MEDICAL_DATA_PATH}")
    # Avisa sobre configurações opcionais
    if not settings.PINECONE_API_KEY:
        print("ℹ️  PINECONE_API_KEY não configurada (opcional para exploração de dados)")
    if not settings.GEMINI_API_KEY:
        print("ℹ️  GEMINI_API_KEY não configurada (opcional para exploração de dados)")

print("\n✅ Bibliotecas importadas com sucesso!")


✅ Configurações validadas com sucesso!
📁 Arquivo de dados: /Users/vitorteixeira/Developer/projects/tech_challenge_fase_3_v2/rag_medical/ori_pqal.json
ℹ️  PINECONE_API_KEY não configurada (opcional para exploração de dados)
ℹ️  GEMINI_API_KEY não configurada (opcional para exploração de dados)

✅ Bibliotecas importadas com sucesso!


In [6]:
# ============================================================================
# ETAPA 1: CARREGAMENTO DO DATASET MÉDICO
# ============================================================================
# Esta etapa carrega o arquivo ori_pqal.json que contém dados médicos
# estruturados do PubMedQA (dataset de perguntas e respostas médicas
# baseadas em evidências científicas)
#
# Estrutura esperada do dataset:
#   {
#     "article_id": {
#       "QUESTION": "Pergunta médica...",
#       "CONTEXTS": ["Contexto 1", "Contexto 2", ...],
#       "LONG_ANSWER": "Resposta longa baseada em evidências...",
#       "MESHES": ["Termo1", "Termo2", ...],
#       "YEAR": "2011",
#       "LABELS": ["BACKGROUND", "RESULTS"],
#       ...
#     },
#     ...
#   }

# Obtém o caminho do arquivo de dados das configurações
data_path = settings.MEDICAL_DATA_PATH

print("=" * 80)
print("📂 CARREGANDO DATASET MÉDICO")
print("=" * 80)
print(f"Caminho do arquivo: {data_path}")
print("-" * 80)

# Carrega o dataset
try:
    raw_data = load_medical_dataset(data_path)
    print(f"✅ Dataset carregado com sucesso!")
except FileNotFoundError as e:
    print(f"❌ Erro: {e}")
    print("\n💡 Dica: Verifique se o caminho está correto no arquivo .env")
    raise
except Exception as e:
    print(f"❌ Erro ao carregar dataset: {e}")
    raise

print(f"Total de entradas médicas: {len(raw_data)}")
print("=" * 80)


📂 CARREGANDO DATASET MÉDICO
Caminho do arquivo: /Users/vitorteixeira/Developer/projects/tech_challenge_fase_3_v2/rag_medical/ori_pqal.json
--------------------------------------------------------------------------------
✅ Dataset carregado com sucesso!
Total de entradas médicas: 1000


In [7]:
# ============================================================================
# ETAPA 2: VALIDAÇÃO DA ESTRUTURA DO DATASET
# ============================================================================
# Esta etapa valida se a estrutura do dataset está correta e identifica
# possíveis problemas ou inconsistências nos dados

print("=" * 80)
print("🔍 VALIDANDO ESTRUTURA DO DATASET")
print("=" * 80)

# Valida estrutura
is_valid, warnings = validate_dataset_structure(raw_data)

if is_valid:
    print("✅ Estrutura do dataset válida!")
else:
    print("⚠️  Avisos encontrados:")
    for warning in warnings:
        print(f"   - {warning}")

print("=" * 80)


🔍 VALIDANDO ESTRUTURA DO DATASET
✅ Estrutura do dataset válida!


In [8]:
# ============================================================================
# ETAPA 3: ESTATÍSTICAS DO DATASET
# ============================================================================
# Esta etapa calcula e exibe estatísticas detalhadas sobre o dataset
# para entender melhor a distribuição e qualidade dos dados

print("=" * 80)
print("📊 ESTATÍSTICAS DO DATASET")
print("=" * 80)

# Calcula estatísticas
stats = get_dataset_stats(raw_data)

print(f"Total de entradas: {stats['total_entries']}")
print(f"Entradas com QUESTION: {stats['entries_with_question']} ({stats['entries_with_question']/stats['total_entries']*100:.1f}%)")
print(f"Entradas com CONTEXTS: {stats['entries_with_contexts']} ({stats['entries_with_contexts']/stats['total_entries']*100:.1f}%)")
print(f"Entradas com LONG_ANSWER: {stats['entries_with_answer']} ({stats['entries_with_answer']/stats['total_entries']*100:.1f}%)")
print(f"\nMédia de contextos por entrada: {stats['avg_contexts_per_entry']:.2f}")
print(f"Tamanho médio das respostas: {stats['avg_answer_length']:.0f} caracteres")

print("=" * 80)


📊 ESTATÍSTICAS DO DATASET
Total de entradas: 1000
Entradas com QUESTION: 1000 (100.0%)
Entradas com CONTEXTS: 1000 (100.0%)
Entradas com LONG_ANSWER: 1000 (100.0%)

Média de contextos por entrada: 3.36
Tamanho médio das respostas: 268 caracteres


In [9]:
# ============================================================================
# ETAPA 4: EXPLORAÇÃO DE EXEMPLOS
# ============================================================================
# Esta etapa exibe exemplos reais de entradas do dataset para entender
# melhor a estrutura e o conteúdo dos dados

print("=" * 80)
print("📄 EXEMPLOS DE ENTRADAS DO DATASET")
print("=" * 80)

# Pega algumas chaves de exemplo
sample_keys = list(raw_data.keys())[:3]

for i, article_id in enumerate(sample_keys, 1):
    entry = raw_data[article_id]
    
    print(f"\n{'='*80}")
    print(f"EXEMPLO {i} - Artigo ID: {article_id}")
    print(f"{'='*80}")
    
    # QUESTION
    question = entry.get('QUESTION', 'N/A')
    print(f"\n📝 QUESTION:")
    print(f"   {question[:200]}{'...' if len(question) > 200 else ''}")
    
    # CONTEXTS
    contexts = entry.get('CONTEXTS', [])
    print(f"\n📚 CONTEXTS ({len(contexts)} contextos):")
    for j, ctx in enumerate(contexts[:2], 1):  # Mostra apenas os 2 primeiros
        print(f"   Contexto {j}: {ctx[:150]}{'...' if len(ctx) > 150 else ''}")
    if len(contexts) > 2:
        print(f"   ... e mais {len(contexts) - 2} contextos")
    
    # LONG_ANSWER
    answer = entry.get('LONG_ANSWER', 'N/A')
    print(f"\n💬 LONG_ANSWER:")
    print(f"   {answer[:200]}{'...' if len(answer) > 200 else ''}")
    
    # MESHES
    meshes = entry.get('MESHES', [])
    print(f"\n🏷️  MESHES ({len(meshes)} termos):")
    if meshes:
        print(f"   {', '.join(meshes[:10])}")
        if len(meshes) > 10:
            print(f"   ... e mais {len(meshes) - 10} termos")
    
    # YEAR
    year = entry.get('YEAR', 'N/A')
    print(f"\n📅 YEAR: {year}")

print(f"\n{'='*80}")
print("✅ Exploração concluída!")
print("=" * 80)


📄 EXEMPLOS DE ENTRADAS DO DATASET

EXEMPLO 1 - Artigo ID: 21645374

📝 QUESTION:
   Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?

📚 CONTEXTS (2 contextos):
   Contexto 1: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in ...
   Contexto 2: The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole...

💬 LONG_ANSWER:
   Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organelle with other organelles during developmental PCD. To the best of ...

🏷️  MESHES (5 termos):
   Alismataceae, Apoptosis, Cell Differentiation, Mitochondria, Plant Leaves

📅 YEAR: 2011

EXEMPLO 2 - Artigo ID: 16418930

📝 QUESTION:
   Landolt C and snellen e acuity: differences in strabismus amblyopia?

📚 CO

## ✅ Conclusão da Etapa 1

Neste notebook você:
- ✅ Carregou o dataset médico do arquivo `ori_pqal.json`
- ✅ Validou a estrutura dos dados
- ✅ Analisou estatísticas do dataset
- ✅ Explorou exemplos de entradas

## 📌 Próximos Passos

Agora você está pronto para o próximo notebook:
- **Notebook 02**: Processamento e limpeza dos dados médicos
- Aplicação de anonimização
- Preparação para embedding e ingestão
